In [2]:
import numpy as np
import pandas as pd
import scipy

from IPython.display import clear_output

import sys
sys.path.append('../../../../Documents/GitHub/gustav/src/')

from gustav import ebi, ncbi, nlm, biogrid, nih
from gustav import publications
from gustav import github
from gustav import access_framework
from gustav import mapper

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../general/src/')
from manuscript import export
from manuscript import inout
from manuscript import datasets
from manuscript import tools

from sklearn.metrics import auc
from scipy.stats import fisher_exact
pd.options.display.precision = 3
pd.options.display.expand_frame_repr = False
pd.options.display.max_columns = 20

In [3]:
taxon = 9606
gene_flavor = 'ou'
ref_genes = datasets.reference_genes(taxon, gene_flavor)
gene_info = ncbi.gene_info(taxon, mode='unambiguous_ensembl')

C:\Users\richa\Documents\GitHub\fmug_analysis\notebooks\../../../../Documents/GitHub/gustav/src\gustav\utils.py:52: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_with_delimiter.drop(column, 1).reset_index(),
C:\Users\richa\Documents\GitHub\fmug_analysis\notebooks\../../../../Documents/GitHub/gustav/src\gustav\utils.py:63: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  joined = joined.drop('helper_index', 1)
C:\Users\richa\Documents\GitHub\fmug_analysis\notebooks\../../../../Documents/GitHub/gustav/src\gustav\utils.py:52: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_with_delimiter.drop(column, 1).reset_index(),
C:\Users\richa\Documents\GitHub\fmug_analysis\notebooks\../../../../Documents/GitHub/gustav/src\gustav\utils.py:

In [4]:
ref_literature = datasets.reference_publications(taxon)

In [5]:
agg = []
literature_flavor = 'pubtator_title_or_abstract_in_any_gene2pubmed_paper'

gene2lit = datasets.reference_gene2lit(
    taxon, 
    literature_flavor)
gene2lit = gene2lit[
    gene2lit['gene_ncbi'].isin(ref_genes) & 
    gene2lit['pubmed_id'].isin(ref_literature)
].copy()

In [6]:
%%time
gene2pubmed = gene2lit #ncbi.gene2pubmed(taxon_ncbi=9606)
large_pubs = gene2pubmed['pubmed_id'].value_counts()[gene2pubmed['pubmed_id'].value_counts() >= 100].index.values
#gene2pubmed = gene2pubmed[~gene2pubmed['pubmed_id'].isin(large_pubs)]

gene2pubmed_dict = gene2pubmed[~gene2pubmed['pubmed_id'].isin(large_pubs)].value_counts('gene_ncbi').to_dict()

CPU times: total: 125 ms
Wall time: 124 ms


# EBI-GWAS

In [7]:
gwas = ebi.gwas(dataset='associations')

In [8]:
gwas['pubmed_id'].value_counts()

35213538    18069
34594039    17797
32888494    16900
30595370    14061
34021172    13546
            ...  
21082022        1
26198393        1
27758888        1
30971808        1
21703177        1
Name: pubmed_id, Length: 5162, dtype: int64

In [9]:
df_array = []

# snp ids
gwas_slice = gwas[gwas['snp_gene_ids'] != None].copy()

gwas_slice['gene_id'] = gwas_slice['snp_gene_ids'].astype(str).str.split(', ')
gwas_slice = gwas_slice.explode('gene_id')
gwas_slice['type'] = 'snp'
df_array.append(gwas_slice)

# downstream ids
gwas_slice = gwas[gwas['downstream_gene_id'] != None]
gwas_slice['gene_id'] = gwas_slice['downstream_gene_id']
gwas_slice['type'] = 'downstream'
df_array.append(gwas_slice)

# upstream ids
gwas_slice = gwas[gwas['upstream_gene_id'] != None]
gwas_slice['gene_id'] = gwas_slice['upstream_gene_id']
gwas_slice['type'] = 'upstream'
df_array.append(gwas_slice)

gwas = pd.concat(df_array).dropna(subset=['gene_id'])
gwas = gwas[(gwas['gene_id'].astype(str) != "None")]

gwas = pd.merge(gwas, gene_info[['gene_ncbi', 'gene_ensembl']], left_on='gene_id', right_on='gene_ensembl')

In [10]:
# All records
gwas['pubmed_id'].value_counts()

35213538    22721
34594039    21592
32888494    19722
34503513    17612
30595370    15661
            ...  
29097723        1
21826085        1
18987618        1
29593015        1
27455349        1
Name: pubmed_id, Length: 4999, dtype: int64

In [11]:
# With hits
gwas[(gwas['pvalue_mlog'] >= -np.log10(5e-8))]['pubmed_id'].value_counts()

35213538    22721
34594039    21592
32888494    19722
34503513    17612
30595370    14928
            ...  
32895509        1
22452962        1
29453196        1
29059430        1
25758998        1
Name: pubmed_id, Length: 3887, dtype: int64

In [12]:
# Hits within gene
gwas[(gwas['pvalue_mlog'] >= -np.log10(5e-8)) & (gwas['type'] == 'snp')]['pubmed_id'].value_counts()

34503513    14772
35213538    12476
34594039    12382
32888494    12120
30595370     8862
            ...  
23297363        1
28468790        1
27601451        1
25338677        1
33542107        1
Name: pubmed_id, Length: 3500, dtype: int64

In [13]:
# Mentions gene in title/abstract
gwas[(gwas['pvalue_mlog'] >= -np.log10(5e-8)) & 
     (gwas['type'] == 'snp') & 
     (gwas['pubmed_id'].isin(gene2pubmed['pubmed_id']))]['pubmed_id'].value_counts()

31761296    337
29124443    112
32581359     67
26414677     45
28754779     41
           ... 
19561606      1
21871595      1
18327257      1
18327256      1
30992453      1
Name: pubmed_id, Length: 651, dtype: int64

In [14]:
gwas_slice = gwas[(gwas['type'] == 'snp') & 
                (gwas['pubmed_id'].isin(gene2pubmed['pubmed_id'])) &
                (gwas['pvalue_mlog'] >= -np.log10(5e-8))].copy()

In [15]:
#gwas_slice['pubmed_id'].drop_duplicates().to_csv('../data/gwas_pubmed_ids.csv', index=False)

In [16]:
%%time
icite = nih.icite(dataset='citations')

CPU times: total: 19 s
Wall time: 19.7 s


In [17]:
def get_genes(gwas, pval_thresh = 5e-8):
    
    prot_genes = set(gene_info[gene_info['type_of_gene'] == 'protein-coding']['gene_ncbi'])
    
    # apply filters and p-value threshold here
    gwas = gwas[(gwas['type'] == 'snp') & 
                (gwas['pubmed_id'].isin(gene2pubmed['pubmed_id'])) &
                (gwas['pvalue_mlog'] >= -np.log10(pval_thresh))].copy()
    
    print(str(len(set(gwas['pubmed_id'].values))) + ' GWAS articles')
    
    icite_slice = icite[icite['referenced'].isin(gwas['pubmed_id'])].copy()
    icite_slice = pd.merge(icite_slice, gene2pubmed, left_on='citing', right_on='pubmed_id')
    print(str(icite_slice['citing'].nunique()) + ' citing articles')

    de_dict = gwas.groupby('pubmed_id')['gene_ncbi'].apply(set).to_dict()
    de_dict_mentioned_genes = (
        gene2pubmed[gene2pubmed['pubmed_id']
                    .isin(gwas['pubmed_id'])]
        .groupby('pubmed_id')['gene_ncbi']
        .apply(set).to_dict()
    )
    de_dict_citations_mentioned_genes = icite_slice.groupby('referenced')['gene_ncbi'].apply(set).to_dict()

    np.random.seed(49)
    n_samp = 100
    de_sets = []
    de_mentioned_sets = []
    de_mentioned_sets_null = []
    de_citations_mentioned_sets = []
    citations_mentioned_sets = []
    result_df_array = []
    citing_articles_array = []
    for comparison_key in np.unique(gwas['pubmed_id'].values):
        
        ### Collect all sets of unique genes
        de_list = de_dict.get(comparison_key) & prot_genes

        if de_dict_mentioned_genes.get(comparison_key):
            de_list_mentioned_genes = de_dict_mentioned_genes.get(comparison_key) & de_list
        else:
            de_list_mentioned_genes = set()
            
        if de_dict_citations_mentioned_genes.get(comparison_key):
            de_list_citations_mentioned_genes = de_dict_citations_mentioned_genes.get(comparison_key) & de_list
        else:
            de_list_citations_mentioned_genes = set()
            
        citing_articles = []
        citing_articles = icite_slice[(icite_slice['referenced'] == comparison_key) &
                                          icite_slice['gene_ncbi'].isin(de_list)]['citing'].values
        for citing_article in citing_articles:
            citing_articles_array.append(citing_article)

        for n_n in range(n_samp):
            de_mentioned_sets_null.append(set(np.random.choice(list(de_list), 
                                                               replace=False, size=len(de_list_mentioned_genes))))

        de_sets.append(de_list)
        de_mentioned_sets.append(de_list_mentioned_genes)
        de_citations_mentioned_sets.append(de_list_citations_mentioned_genes)
        
        ### Impute transition probabilities
        result_df = pd.DataFrame()
        result_df['gene_ncbi'] = list(de_list)
        result_df['comparison'] = comparison_key
        result_df['mentioned'] = result_df['gene_ncbi'].isin(de_list_mentioned_genes)
        result_df['mentioned_citations'] = result_df['gene_ncbi'].isin(de_list_citations_mentioned_genes)
        result_df_array.append(result_df)
        
    ### Collect unique genes
    de_list = set().union(*de_sets)
    de_list_mentioned = set().union(*de_mentioned_sets)
    de_list_mentioned_genes_null = set().union(*de_mentioned_sets_null)
    de_list_citations_mentioned = set().union(*de_citations_mentioned_sets)
    
    ### Collect data for each set
    result_df = pd.concat(result_df_array)
    
    helper = result_df.groupby('comparison').sum()#['mentioned']
    prohib_ids = helper[helper['mentioned'] == 0].index.values
    result_df = result_df[~result_df['comparison'].isin(prohib_ids)]
    result_df['hit'] = True
    result_df = result_df.rename(columns={'comparison':'pubmed_id'})
    print(str(len(set(result_df['pubmed_id'].values))) + ' valid GWAS articles')
    print(str(len(np.unique(citing_articles_array))) + ' valid citing articles')
    
    result_df.to_csv('../data/gwas_all_hits_' + str(pval_thresh) + '.csv', index=False)
    return result_df

In [18]:
#result_df = get_genes(gwas, pval_thresh=1e-5)
result_df = get_genes(gwas, pval_thresh=5e-8)
#result_df = get_genes(gwas, pval_thresh=1e-8)
#result_df = get_genes(gwas, pval_thresh=1e-9)
#result_df = get_genes(gwas, pval_thresh=1e-10)

651 GWAS articles
9451 citing articles
450 valid GWAS articles
3524 valid citing articles


In [19]:
print(result_df[result_df['hit']]['gene_ncbi'].nunique())

print(result_df[result_df['mentioned']]['gene_ncbi'].nunique())

print(result_df[result_df['mentioned_citations']]['gene_ncbi'].nunique())

1043
413
319
